In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pl
pl.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl
import dataviz as dv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
verbose = True

PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\"
RAW = PATH + "Raw"

In [21]:
file = PATH + "PreProcessed\\sessions.xlsx"
df = pd.read_excel(file)
print(len(df), 'sessions read')
df = df.sort_values(by='sess_id', ascending=True)
df.head(3)

482 sessions read


,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
0,focused-active,2862,2975,16632.760739,68.597096,7614.272703,0.457788,31.402904,60.349820,1061.744934,...,sit,25,239.316870,233.739654,2017-09-29 12:25:28,2017-09-29 13:15:03,29488.443381,0,5241.409939,0
1,leisure,484,520,13446.858350,76.104324,4222.122427,0.313986,23.895676,56.160275,1096.966942,...,sit,0,215.915390,172.462601,2017-09-29 13:20:00,2017-09-29 13:28:40,20709.546077,0,3040.565300,1
2,eat,381,259,851.708553,53.858480,729.673897,0.856718,46.141520,87.674022,693.028871,...,sit,2,65.153745,66.925227,2017-09-29 13:46:19,2017-09-29 13:50:38,2276.656888,0,695.274438,2


In [22]:
fdf = pd.read_excel(PATH+ 'PreProcessed/df_60_0.xlsx')
#fdf[(fdf.start > datetime(2017,10,14,11,48)) & (fdf.stop <  datetime(2017,10,14,14))][['start', 'stop', 'activity', 'sess', 'user']]

## Plotting random examples of RR time series for each type of activity

In [27]:
for act in df['activity'].unique():
    v = df[df['activity'] == act].sample(1)
    for i in range(1):
        dv.full_plot_sess(v.iloc[0], fdf, RAW)

SESSION ID:  419
ACTIVITY:  focused-active
USER:  3
TIME:  2017-11-01 13:25:29


SESSION ID:  427
ACTIVITY:  leisure
USER:  4
TIME:  2017-12-20 21:36:05


SESSION ID:  109
ACTIVITY:  eat
USER:  0
TIME:  2017-11-05 12:40:44


SESSION ID:  21
ACTIVITY:  sleep
USER:  0
TIME:  2017-10-04 11:24:00


SESSION ID:  124
ACTIVITY:  movement
USER:  0
TIME:  2017-11-07 21:31:08


...

SESSION ID:  97
ACTIVITY:  exercise
USER:  0
TIME:  2017-11-01 07:27:13


SESSION ID:  101
ACTIVITY:  commute
USER:  0
TIME:  2017-11-01 10:25:52


SESSION ID:  217
ACTIVITY:  train-baseline
USER:  0
TIME:  2018-06-18 17:40:25


SESSION ID:  299
ACTIVITY:  train-focus
USER:  1
TIME:  2018-06-21 17:04:23


SESSION ID:  209
ACTIVITY:  train-breathe
USER:  0
TIME:  2018-06-18 11:10:18


## Assessing how features evolve within a session
---

### Plotting a specific session

In [7]:
dv.full_plot_sess(df.iloc[111], fdf, RAW)

SESSION ID:  111
ACTIVITY:  exercise
USER:  0
TIME:  2017-11-01 11:29:55


### Helper to choose sessions

In [16]:
df.loc[(df.activity == 'eat') & (df.user.isin([0]))][['user', 'activity', 'start', 'duration', 'beatscount', 'rmssd', 'pnn50']].sort_values(by='pnn50', ascending=False)

,user,activity,start,duration,beatscount,rmssd,pnn50
2,0,eat,2017-09-29 13:32:20,202,269,158.107901,46.641791
16,0,eat,2017-10-01 13:37:53,571,858,142.113592,33.488915
25,0,eat,2017-10-04 12:38:34,626,948,191.392030,29.461457
15,0,eat,2017-09-30 13:30:45,145,247,147.821484,26.829268
18,0,eat,2017-10-03 22:56:19,640,972,120.700624,23.583934
3,0,eat,2017-09-29 13:46:19,259,381,65.153745,13.684211
144,0,eat,2017-11-15 12:12:44,513,854,88.914662,12.075029
53,0,eat,2017-10-12 13:40:00,600,921,60.378678,11.413043
118,0,eat,2017-11-05 12:40:44,571,1007,70.780525,10.437376
158,0,eat,2017-11-23 14:29:00,466,784,40.808997,9.578544


### Plotting all sessions in a filter

In [17]:
v = df[(df['activity'] == 'eat') & (df.user.isin([0])) & (df.rmssd > 64) & (df.pnn50 > 15)]
for i in range(len(v)):
    dv.full_plot_sess(v.iloc[i], fdf, RAW)

SESSION ID:  2
ACTIVITY:  eat
USER:  0
TIME:  2017-09-29 13:32:20


SESSION ID:  15
ACTIVITY:  eat
USER:  0
TIME:  2017-09-30 13:30:45


SESSION ID:  16
ACTIVITY:  eat
USER:  0
TIME:  2017-10-01 13:37:53


SESSION ID:  18
ACTIVITY:  eat
USER:  0
TIME:  2017-10-03 22:56:19


SESSION ID:  25
ACTIVITY:  eat
USER:  0
TIME:  2017-10-04 12:38:34


In [29]:
fdf[fdf.sess.isin([217,209,299])]

,activity,beatcount,f_id,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,pnn50,posture,rmssd,sdnn,sess,start,stop,total_power,user,vlf
13505,train-breathe,77,0,223.466260,7.863887,2618.210662,11.716358,92.136113,77.288438,784.740260,...,19.736842,sit,57.267264,74.510226,209,2018-06-18 11:10:18,2018-06-18 11:11:18,2920.133800,0,78.456878
13506,train-breathe,76,1,258.939920,8.411179,2819.581084,10.888939,91.588821,75.812645,795.171053,...,14.666667,sit,32.434961,54.738503,209,2018-06-18 11:11:18,2018-06-18 11:12:18,3253.679545,0,175.158540
13521,train-baseline,73,0,7102.825684,77.647002,2044.759554,0.287880,22.352998,79.248559,808.287671,...,58.333333,sit,190.355771,170.525453,217,2018-06-18 17:40:25,2018-06-18 17:41:25,13377.204693,0,4229.619455
13522,train-baseline,73,1,680.387534,76.490871,209.114086,0.307346,23.509129,72.613516,834.753425,...,33.333333,sit,95.268480,94.122376,217,2018-06-18 17:41:25,2018-06-18 17:42:25,3133.391690,0,2243.890070
15666,train-focus,86,0,141.159944,35.132697,260.630858,1.846351,64.867303,85.547141,703.046512,...,1.176471,sit,16.608290,34.585242,299,2018-06-21 17:04:23,2018-06-21 17:05:23,557.734849,1,155.944048
15667,train-focus,86,1,59.042466,20.336226,231.289016,3.917333,79.663774,84.318118,712.883721,...,1.176471,sit,17.959104,30.312264,299,2018-06-21 17:05:23,2018-06-21 17:06:23,352.177821,1,61.846339
